# 创建部署

让我们部署在模块 5 中构建的 `task_maistro` 应用。

## 代码结构

要创建 LangGraph Platform 部署，需要提供[如下信息](https://langchain-ai.github.io/langgraph/concepts/application_structure/)：

* [LangGraph API 配置文件](https://langchain-ai.github.io/langgraph/concepts/application_structure/#configuration-file) —— `langgraph.json`
* 实现应用逻辑的图（例如 `task_maistro.py`）
* 指定运行应用所需依赖的文件 —— `requirements.txt`
* 应用运行所需的环境变量 —— `.env` 或 `docker-compose.yml`

这些内容我们已经放在 `module-6/deployment` 目录里了！

## CLI

[LangGraph CLI](https://langchain-ai.github.io/langgraph/concepts/langgraph_cli/) 是用于创建 LangGraph Platform 部署的命令行工具。


In [1]:
%%capture --no-stderr
%pip install -U langgraph-cli

若要创建[自托管部署](https://langchain-ai.github.io/langgraph/how-tos/deploy-self-hosted/#how-to-do-a-self-hosted-deployment-of-langgraph)，可以按照以下步骤进行。

### 构建 LangGraph Server 的 Docker 镜像

我们先使用 langgraph CLI 为 [LangGraph Server](https://docs.google.com/presentation/d/18MwIaNR2m4Oba6roK_2VQcBE_8Jq_SI7VHTXJdl7raU/edit#slide=id.g313fb160676_0_32) 创建 Docker 镜像。

这样就能把图和依赖一起打包到镜像里。

Docker 镜像是 Docker 容器的模板，包含运行应用所需的代码与依赖。

确认已经安装 [Docker](https://docs.docker.com/engine/install/)，然后运行以下命令生成名为 `my-image` 的镜像：

```
$ cd module-6/deployment
$ langgraph build -t my-image
```

### 设置 Redis 与 PostgreSQL

如果你已经在本地或其他服务器上运行 Redis 和 PostgreSQL，可[单独启动 LangGraph Server 容器](https://langchain-ai.github.io/langgraph/how-tos/deploy-self-hosted/#running-the-application-locally)，并为其提供 Redis 与 PostgreSQL 的 URI：

```
docker run \
    --env-file .env \
    -p 8123:8000 \
    -e REDIS_URI="foo" \
    -e DATABASE_URI="bar" \
    -e LANGSMITH_API_KEY="baz" \
    my-image
```

或者，可以使用提供的 `docker-compose.yml` 文件创建三个独立容器：

* `langgraph-redis`：基于官方 Redis 镜像创建新容器。
* `langgraph-postgres`：基于官方 Postgres 镜像创建新容器。
* `langgraph-api`：基于你预先构建的镜像创建容器。

只需复制 `docker-compose-example.yml`，并添加以下环境变量，即可运行部署好的 `task_maistro` 应用：

* `IMAGE_NAME`（例如 `my-image`）
* `LANGSMITH_API_KEY`
* `OPENAI_API_KEY`

然后[启动部署](https://langchain-ai.github.io/langgraph/how-tos/deploy-self-hosted/#using-docker-compose)：

```
$ cd module-6/deployment
$ docker compose up
```


只是想临时停：docker compose stop

- `docker compose ps`: `Exited`

- 再启动：`docker compose start`（或 `docker compose up -d`）

想彻底停掉并清理网络：docker compose down

- 再启动：`docker compose up -d`

想连数据卷也一并清掉：docker compose down -v